# 4. Testes de Hipóteses nos Dados PISA

Este notebook realiza testes de hipóteses para investigar relações significativas nos dados.

## Objetivos
1. Testar diferenças entre grupos
2. Avaliar correlações significativas
3. Investigar tendências temporais
4. Analisar efeitos de políticas educacionais
5. Validar pressupostos estatísticos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

%matplotlib inline
plt.style.use('seaborn')
pd.set_option('display.max_columns', None)

## 1. Carregamento e Preparação dos Dados

In [ ]:
# Carrega dados
df = pd.read_csv('../data/economics_and_education_dataset_CSV.csv')

# Remove linhas com valores ausentes nas variáveis principais
main_vars = ['rating', 'expenditure_on _education_pct_gdp', 'gdp_per_capita_ppp', 'gini_index']
df_clean = df.dropna(subset=main_vars)

# Cria categorias para análise
df_clean['gdp_category'] = pd.qcut(df_clean['gdp_per_capita_ppp'], q=3, labels=['Low', 'Medium', 'High'])
df_clean['education_spending_category'] = pd.qcut(df_clean['expenditure_on _education_pct_gdp'], q=3, labels=['Low', 'Medium', 'High'])

print('Shape após limpeza:', df_clean.shape)

## 2. Teste de Hipótese 1: Diferença entre Sexos

In [ ]:
def analyze_gender_difference(data):
    """Analisa diferença de desempenho entre sexos."""
    # Separa grupos
    boys = data[data['sex'] == 'BOY']['rating']
    girls = data[data['sex'] == 'GIRL']['rating']
    
    # Teste t
    t_stat, p_val = stats.ttest_ind(boys, girls)
    
    # Estatísticas descritivas
    stats_df = pd.DataFrame({
        'Média': [boys.mean(), girls.mean()],
        'Desvio Padrão': [boys.std(), girls.std()],
        'N': [len(boys), len(girls)]
    }, index=['Meninos', 'Meninas'])
    
    # Visualização
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='sex', y='rating', data=data)
    plt.title('Distribuição das Notas por Sexo')
    plt.show()
    
    print('Estatísticas Descritivas:')
    print(stats_df.round(2))
    print('\nResultados do Teste t:')
    print(f'T-statistic: {t_stat:.4f}')
    print(f'P-valor: {p_val:.4f}')
    print('Conclusão:', 'Diferença significativa' if p_val < 0.05 else 'Sem diferença significativa')

analyze_gender_difference(df_clean)

## 3. Teste de Hipótese 2: Efeito do PIB

In [ ]:
def analyze_gdp_effect(data):
    """Analisa efeito do PIB no desempenho."""
    # ANOVA
    groups = [group['rating'].values for name, group in data.groupby('gdp_category')]
    f_stat, p_val = stats.f_oneway(*groups)
    
    # Teste post-hoc
    tukey = pairwise_tukeyhsd(data['rating'], data['gdp_category'])
    
    # Visualização
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='gdp_category', y='rating', data=data)
    plt.title('Notas por Categoria de PIB')
    plt.show()
    
    print('Resultados da ANOVA:')
    print(f'F-statistic: {f_stat:.4f}')
    print(f'P-valor: {p_val:.4f}')
    print('\nAnálise Post-hoc (Tukey):')
    print(tukey)

analyze_gdp_effect(df_clean)

## 4. Teste de Hipótese 3: Correlação com Gastos em Educação

In [ ]:
def analyze_education_spending(data):
    """Analisa relação entre gastos em educação e desempenho."""
    # Correlação
    corr, p_val = stats.pearsonr(data['expenditure_on _education_pct_gdp'], data['rating'])
    
    # Regressão simples
    X = sm.add_constant(data['expenditure_on _education_pct_gdp'])
    model = sm.OLS(data['rating'], X).fit()
    
    # Visualização
    plt.figure(figsize=(10, 6))
    sns.regplot(x='expenditure_on _education_pct_gdp', y='rating', data=data)
    plt.title('Relação entre Gastos em Educação e Notas')
    plt.xlabel('Gastos em Educação (% PIB)')
    plt.ylabel('Nota PISA')
    plt.show()
    
    print('Análise de Correlação:')
    print(f'Correlação: {corr:.4f}')
    print(f'P-valor: {p_val:.4f}')
    print('\nResultados da Regressão:')
    print(model.summary().tables[1])

analyze_education_spending(df_clean)

## 5. Teste de Hipótese 4: Tendência Temporal

In [ ]:
def analyze_temporal_trend(data):
    """Analisa tendência temporal nas notas."""
    # Calcula médias anuais
    yearly_means = data.groupby('time')['rating'].mean()
    
    # Teste de tendência
    slope, intercept, r_value, p_value, std_err = stats.linregress(range(len(yearly_means)), yearly_means)
    
    # Mann-Kendall test
    from scipy.stats import kendalltau
    tau, p_val = kendalltau(yearly_means.index, yearly_means.values)
    
    # Visualização
    plt.figure(figsize=(10, 6))
    plt.plot(yearly_means.index, yearly_means.values, 'o-')
    plt.plot(yearly_means.index, intercept + slope * np.arange(len(yearly_means)), 'r--')
    plt.title('Tendência Temporal das Notas PISA')
    plt.xlabel('Ano')
    plt.ylabel('Nota Média')
    plt.grid(True)
    plt.show()
    
    print('Análise de Tendência Linear:')
    print(f'Inclinação: {slope:.4f}')
    print(f'R²: {r_value**2:.4f}')
    print(f'P-valor: {p_value:.4f}')
    print('\nTeste de Mann-Kendall:')
    print(f'Tau: {tau:.4f}')
    print(f'P-valor: {p_val:.4f}')

analyze_temporal_trend(df_clean)

## 6. Teste de Hipótese 5: Interação entre PIB e Gastos em Educação

In [ ]:
def analyze_gdp_education_interaction(data):
    """Analisa interação entre PIB e gastos em educação."""
    # ANOVA de dois fatores
    from statsmodels.stats.anova import anova_lm
    from statsmodels.formula.api import ols
    
    model = ols('rating ~ C(gdp_category) + C(education_spending_category) + C(gdp_category):C(education_spending_category)',
                data=data).fit()
    anova_table = anova_lm(model, typ=2)
    
    # Médias por grupo
    means = data.groupby(['gdp_category', 'education_spending_category'])['rating'].mean().unstack()
    
    # Visualização
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Boxplot
    sns.boxplot(x='gdp_category', y='rating', hue='education_spending_category', data=data, ax=ax1)
    ax1.set_title('Distribuição das Notas por Categoria')
    
    # Interaction plot
    means.plot(marker='o', ax=ax2)
    ax2.set_title('Gráfico de Interação')
    ax2.set_xlabel('Categoria de PIB')
    ax2.set_ylabel('Nota Média')
    
    plt.tight_layout()
    plt.show()
    
    print('ANOVA de Dois Fatores:')
    print(anova_table)
    print('\nMédias por Grupo:')
    print(means.round(2))

analyze_gdp_education_interaction(df_clean)